### 비마이펫 사이트 크롤링 코드

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import time

# 셀레니움 기본 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

# 페이지 열기
url = 'https://bemypet.kr/content/doglab?p=1'
driver.get(url)
time.sleep(3)

# soup 생성
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# 기사 블록 추출 (class가 변하므로 prefix로 처리)
articles = soup.find_all("div", class_=re.compile("^textContent_textContentBox"))

print("✅ 총 기사 수:", len(articles))

for a in articles:
    title = a.find("div", class_=re.compile("^textContent_textContentTitle"))
    summary = a.find("div", class_=re.compile("^textContent_textContentText"))
    
    print("📌 제목:", title.get_text(strip=True) if title else "없음")
    print("📝 요약:", summary.get_text(strip=True)[:100] if summary else "없음")
    print("-" * 40)


✅ 총 기사 수: 10
📌 제목: 아기 강아지가 특히 취약한 질병 4가지와 예방법
📝 요약: 초보 보호자라면 꼭 알아야 할 생후 초기 건강관리 팁



아기 강아지는 작고 여린 만큼 건강 관리에 특별한 주의가 필요합니다. 아직 면역 체계가 완전히 발달하지 않았기 때문에, 
----------------------------------------
📌 제목: 강아지 고양이 소변검사, 받아야 하는 이유와 주요 항목
📝 요약: 최근 반려동물 보호자들 사이에서 '반려동물 소변검사 셀프 키트'가 인기를 끌고 있습니다.집에서 간편하게 반려견이나 고양이의 소변을 채취하고, 검사지에 따라 색 변화로 건강 상태를 
----------------------------------------
📌 제목: 강아지 음수량 늘리기 방법 5가지｜실내에서 쉽게 실천하는 꿀팁 모음
📝 요약: 강아지의 건강을 지키는 가장 간단한 방법 중 하나가 충분한 물을 마시게 하는 것이라는 사실, 알고 계셨나요?특히 1~7세의 성견은 신체 기능이 활발히 유지되고 있는 시기이지만, 자
----------------------------------------
📌 제목: 강아지 쿠싱 증후군, 조용한 암살자를 아시나요?
📝 요약: 강아지 쿠싱 증후군(Cushing's syndrome)은 '조용한 암살자'라는 별명이 붙을 정도로 서서히, 그러나 치명적으로 강아지의 건강을 위협하는 질병입니다. 주로 노령견에게 
----------------------------------------
📌 제목: 노령견의 면역력, 어떻게 지켜야 할까요?
📝 요약: 면역력이 약해지는 노령견을 위한 생활 습관 가이드



나이가 들면서 강아지의 몸에는 다양한 변화가 나타나기 시작합니다. 그중 가장 중요한 변화 중 하나는 바로 면역력 저하입니다.
----------------------------------------
📌 제목: 봄철 강아지 발바닥 관리, 지금부터 시작해야 하는 이유
📝 요약: 따뜻한 봄이 찾아오면 강아지와 함께하는 산책이 더욱

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import csv
import time

# 셀레니움 설정
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 1페이지 접속
driver.get('https://bemypet.kr/content/doglab?p=1')
time.sleep(2)

# ✅ 'Go to last page' 버튼 클릭 → 마지막 페이지로 이동
try:
    last_button = driver.find_element(By.XPATH, '//a[@aria-label="Go to last page"]')
    last_button.click()
    time.sleep(2)
except Exception as e:
    print("❌ 마지막 페이지 이동 실패:", e)
    driver.quit()
    exit()

# ✅ 마지막 페이지 번호 추출
soup = BeautifulSoup(driver.page_source, 'html.parser')
pagination = soup.select("ul.pagination li a")
page_numbers = [int(a.text.strip()) for a in pagination if a.text.strip().isdigit()]
last_page = max(page_numbers) if page_numbers else 1

print(f"📄 마지막 페이지: {last_page}")

# ✅ 수집 시작
results = []

for page in range(1, last_page + 1):
    print(f"\n📍 페이지 {page} 수집 중...")
    driver.get(f'https://bemypet.kr/content/doglab?p={page}')
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    articles = soup.find_all("div", class_=re.compile("^textContent_textContentBox"))

    for a in articles:
        title = a.find("div", class_=re.compile("^textContent_textContentTitle"))
        summary = a.find("div", class_=re.compile("^textContent_textContentText"))

        if title and summary:
            results.append({
                "title": title.get_text(strip=True),
                "summary": summary.get_text(strip=True),
                "page_num": page
            })

    print(f"✅ 누적 수집 기사 수: {len(results)}")

    # 중간 저장
    if page % 10 == 0 or page == last_page:
        with open('doglab_partial_final.csv', 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=["title", "summary", "page_num"])
            writer.writeheader()
            writer.writerows(results)
        print(f"💾 임시 저장 완료 (페이지 {page})")

driver.quit()

# ✅ 최종 저장
with open('doglab_all_articles.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=["title", "summary", "page_num"])
    writer.writeheader()
    writer.writerows(results)

print(f"\n✅ 전체 수집 완료: {len(results)}개 기사 저장됨 → doglab_all_articles.csv")


📄 마지막 페이지: 127

📍 페이지 1 수집 중...
✅ 누적 수집 기사 수: 10

📍 페이지 2 수집 중...
✅ 누적 수집 기사 수: 20

📍 페이지 3 수집 중...
✅ 누적 수집 기사 수: 30

📍 페이지 4 수집 중...
✅ 누적 수집 기사 수: 40

📍 페이지 5 수집 중...
✅ 누적 수집 기사 수: 50

📍 페이지 6 수집 중...
✅ 누적 수집 기사 수: 60

📍 페이지 7 수집 중...
✅ 누적 수집 기사 수: 70

📍 페이지 8 수집 중...
✅ 누적 수집 기사 수: 80

📍 페이지 9 수집 중...
✅ 누적 수집 기사 수: 90

📍 페이지 10 수집 중...
✅ 누적 수집 기사 수: 100
💾 임시 저장 완료 (페이지 10)

📍 페이지 11 수집 중...
✅ 누적 수집 기사 수: 110

📍 페이지 12 수집 중...
✅ 누적 수집 기사 수: 120

📍 페이지 13 수집 중...
✅ 누적 수집 기사 수: 130

📍 페이지 14 수집 중...
✅ 누적 수집 기사 수: 140

📍 페이지 15 수집 중...
✅ 누적 수집 기사 수: 150

📍 페이지 16 수집 중...
✅ 누적 수집 기사 수: 160

📍 페이지 17 수집 중...
✅ 누적 수집 기사 수: 170

📍 페이지 18 수집 중...
✅ 누적 수집 기사 수: 180

📍 페이지 19 수집 중...
✅ 누적 수집 기사 수: 190

📍 페이지 20 수집 중...
✅ 누적 수집 기사 수: 200
💾 임시 저장 완료 (페이지 20)

📍 페이지 21 수집 중...
✅ 누적 수집 기사 수: 210

📍 페이지 22 수집 중...
✅ 누적 수집 기사 수: 220

📍 페이지 23 수집 중...
✅ 누적 수집 기사 수: 230

📍 페이지 24 수집 중...
✅ 누적 수집 기사 수: 240

📍 페이지 25 수집 중...
✅ 누적 수집 기사 수: 250

📍 페이지 26 수집 중...
✅ 누적 수집 기사 수: 260

📍 페이지 27 수집 중...
✅ 누적 수집 기

----

### 콘텐츠용 구조화 코드

In [8]:
import requests
from bs4 import BeautifulSoup
import json

def parse_mypetlife_article(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # 제목
    title_tag = soup.find("h1", class_="entry-title")
    title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

    # 태그
    tag_div = soup.select_one(".post-bottom-tags .tagcloud")
    tags = [a.get_text(strip=True) for a in tag_div.find_all("a")] if tag_div else []

    # 본문
    content_blocks = []
    content_div = soup.find("div", class_="entry-content")

    if not content_div:
        print("❌ 본문을 찾을 수 없음")
        return None

    for elem in content_div.find_all(["p", "h2", "h3", "h4", "ul", "ol", "figure"], recursive=True):
        tag = elem.name

        # 단락
        if tag == "p":
            text = elem.get_text(strip=True)
            if text:
                content_blocks.append({"type": "paragraph", "text": text})

        # 제목 계층
        elif tag == "h2":
            content_blocks.append({"type": "heading", "text": elem.get_text(strip=True)})
        elif tag in ["h3", "h4"]:
            text = elem.get_text(strip=True)
            if "✔️" in text or "팁" in text:
                content_blocks.append({"type": "tip_heading", "text": text})
            else:
                content_blocks.append({"type": "subheading", "text": text})

        # 리스트
        elif tag in ["ul", "ol"]:
            items = [li.get_text(strip=True) for li in elem.find_all("li")]
            if items:
                content_blocks.append({"type": "list", "items": items})

        # 이미지
        elif tag == "figure":
            img = elem.find("img")
            if img and img.get("src"):
                content_blocks.append({"type": "image", "url": img["src"]})

    return {
        "url": url,
        "title": title,
        "tags": tags,
        "content": content_blocks
    }


# ✅ 실행 예시
if __name__ == "__main__":
    test_url = "https://mypetlife.co.kr/152998/"
    result = parse_mypetlife_article(test_url)

    if result:
        # 보기 좋게 출력
        print("📰 제목:", result['title'])
        print("🏷️ 태그:", ", ".join(result['tags']))
        print("📦 콘텐츠 블록 수:", len(result['content']))

        # JSON 저장 (선택)
        with open("mypetlife_structured.json", "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
        print("✅ JSON 저장 완료 → mypetlife_structured.json")


📰 제목: 강아지 공복토가 계속될 때, 보호자가 꼭 알아야 할 7가지
🏷️ 태그: 강아지 건강, 강아지 공복시간, 강아지 공복토, 강아지 공복토 계속, 강아지 공복토 색깔, 강아지 공복토 원인, 강아지 공복토 유산균, 강아지 공복토 피, 강아지 공복토 후 사료, 강아지 영양제, 강아지 질병, 강아지 행동
📦 콘텐츠 블록 수: 57
✅ JSON 저장 완료 → mypetlife_structured.json


In [11]:
import requests
from bs4 import BeautifulSoup
import json

def parse_mypetlife_article(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # 제목
    title_tag = soup.find("h1", class_="entry-title")
    title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

    # 태그
    tag_div = soup.select_one(".post-bottom-tags .tagcloud")
    tags = [a.get_text(strip=True) for a in tag_div.find_all("a")] if tag_div else []

    # 본문
    content_blocks = []
    content_div = soup.find("div", class_="entry-content")
    if not content_div:
        print("❌ 본문을 찾을 수 없음")
        return None

    # 필요한 태그만 필터링 (whitelist)
    for elem in content_div.find_all(recursive=True):
        tag = elem.name

        # ✅ 태그 허용 목록
        if tag not in {"p", "h2", "h3", "h4", "ul", "ol", "figure"}:
            continue

        # ⛔️ 클래스 기반 광고/배너/삽입 콘텐츠 제외
        if elem.get("class") and any("ads" in cls or "google" in cls or "stream" in cls for cls in elem.get("class")):
            continue

        # ⛔️ iframe, noscript 포함 요소 제외
        if elem.find("iframe") or elem.find("noscript"):
            continue

        text = elem.get_text(strip=True)

        # ✅ paragraph
        if tag == "p" and text:
            content_blocks.append({"type": "paragraph", "text": text})

        # ✅ heading
        elif tag == "h2":
            content_blocks.append({"type": "heading", "text": text})

        # ✅ subheading or tip_heading
        elif tag in ["h3", "h4"]:
            if "✔️" in text or "팁" in text:
                content_blocks.append({"type": "tip_heading", "text": text})
            else:
                content_blocks.append({"type": "subheading", "text": text})

        # ✅ list
        elif tag in ["ul", "ol"]:
            items = [li.get_text(strip=True) for li in elem.find_all("li") if li.get_text(strip=True)]
            if items:
                content_blocks.append({"type": "list", "items": items})

        # ✅ image (SVG 제외)
        elif tag == "figure":
            img = elem.find("img")
            if img and img.get("src") and not img["src"].startswith("data:image/svg+xml"):
                content_blocks.append({"type": "image", "url": img["src"]})

    return {
        "url": url,
        "title": title,
        "tags": tags,
        "content": content_blocks
    }

# ✅ 테스트 실행
if __name__ == "__main__":
    test_url = "https://mypetlife.co.kr/153224/"
    result = parse_mypetlife_article(test_url)

    if result:
        print("📰 제목:", result['title'])
        print("🏷️ 태그:", ", ".join(result['tags']))
        print("📦 콘텐츠 블록 수:", len(result['content']))

        with open("mypetlife_structured2.json", "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
        print("✅ JSON 저장 완료 → mypetlife_structured2.json")


📰 제목: 아기 강아지가 특히 취약한 질병 4가지와 예방법
🏷️ 태그: 강아지 예방접종, 강아지 접종, 강아지 켄넬코프, 강아지 코로나, 강아지 파보, 강아지 홍역, 아기강아지 접종 주기, 아기강아지 질병
📦 콘텐츠 블록 수: 39
✅ JSON 저장 완료 → mypetlife_structured2.json


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

driver.get("https://bemypet.kr/content/doglab?p=1")
time.sleep(2)

all_article_urls = []
page_limit = 3
current_page = 1

while current_page <= page_limit:
    print(f"\n📄 페이지 {current_page} 처리 중...")

    try:
        cards = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "textContent_textContentBox__9iWXJ"))
        )
    except Exception as e:
        print(f"❌ 카드 로딩 실패 (페이지 {current_page}):", e)
        break

    for i in range(len(cards)):
        try:
            main_window = driver.current_window_handle

            # ✅ 광고 방지용 스크롤 후 클릭 시도
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cards[i])
            time.sleep(0.5)
            try:
                cards[i].click()
            except:
                driver.execute_script("arguments[0].click();", cards[i])  # JS 클릭 대체

            time.sleep(2)

            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_window = [w for w in driver.window_handles if w != main_window][0]
            driver.switch_to.window(new_window)

            url = driver.current_url
            print("🔗 기사 URL:", url)
            if "mypetlife.co.kr" in url:
                all_article_urls.append(url)

            driver.close()
            driver.switch_to.window(main_window)

            cards = driver.find_elements(By.CLASS_NAME, "textContent_textContentBox__9iWXJ")

        except Exception as e:
            print(f"❌ 카드 {i} 클릭 실패:", e)
            continue

    # ✅ 다음 페이지 버튼 클릭
    try:
        next_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@aria-label="Go to next page"]'))
        )
        next_btn.click()
        current_page += 1
        time.sleep(2)
    except Exception as e:
        print("❌ 다음 페이지 클릭 실패:", e)
        break

driver.quit()

# ✅ 저장
print(f"\n✅ 테스트 수집 완료: 총 {len(all_article_urls)}개 URL")
with open("test_mypet_urls.json", "w", encoding="utf-8") as f:
    json.dump(all_article_urls, f, indent=2, ensure_ascii=False)



📄 페이지 1 처리 중...
🔗 기사 URL: https://mypetlife.co.kr/153331/
🔗 기사 URL: https://mypetlife.co.kr/153224/
🔗 기사 URL: https://mypetlife.co.kr/153162/
🔗 기사 URL: https://mypetlife.co.kr/153151/
🔗 기사 URL: https://mypetlife.co.kr/153128/
🔗 기사 URL: https://mypetlife.co.kr/153105/
🔗 기사 URL: https://mypetlife.co.kr/153094/
🔗 기사 URL: https://mypetlife.co.kr/153055/
🔗 기사 URL: https://mypetlife.co.kr/153040/
🔗 기사 URL: https://mypetlife.co.kr/153032/

📄 페이지 2 처리 중...
🔗 기사 URL: https://mypetlife.co.kr/153017/
🔗 기사 URL: https://mypetlife.co.kr/152998/
🔗 기사 URL: https://mypetlife.co.kr/152914/
🔗 기사 URL: https://mypetlife.co.kr/152908/
🔗 기사 URL: https://mypetlife.co.kr/152899/
🔗 기사 URL: https://mypetlife.co.kr/152892/
🔗 기사 URL: https://mypetlife.co.kr/152884/
🔗 기사 URL: https://mypetlife.co.kr/152861/
🔗 기사 URL: https://mypetlife.co.kr/152837/
🔗 기사 URL: https://mypetlife.co.kr/152807/

📄 페이지 3 처리 중...
🔗 기사 URL: https://mypetlife.co.kr/152795/
🔗 기사 URL: https://mypetlife.co.kr/152788/
🔗 기사 URL: https://mypetli

In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

driver.get("https://bemypet.kr/content/doglab?p=1")
time.sleep(2)

all_article_urls = []
page_limit = 127  # 전체 페이지 수
current_page = 1

while current_page <= page_limit:
    print(f"\n📄 페이지 {current_page} 처리 중...")

    try:
        cards = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "textContent_textContentBox__9iWXJ"))
        )
    except Exception as e:
        print(f"❌ 카드 로딩 실패 (페이지 {current_page}):", e)
        break

    for i in range(len(cards)):
        try:
            main_window = driver.current_window_handle

            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cards[i])
            time.sleep(0.5)
            try:
                cards[i].click()
            except:
                driver.execute_script("arguments[0].click();", cards[i])

            time.sleep(2)

            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_window = [w for w in driver.window_handles if w != main_window][0]
            driver.switch_to.window(new_window)

            url = driver.current_url
            print("🔗 기사 URL:", url)
            if "mypetlife.co.kr" in url:
                all_article_urls.append(url)

            driver.close()
            driver.switch_to.window(main_window)

            cards = driver.find_elements(By.CLASS_NAME, "textContent_textContentBox__9iWXJ")

        except Exception as e:
            print(f"❌ 카드 {i} 클릭 실패:", e)
            continue

    # ✅ 다음 페이지로 이동
    try:
        next_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@aria-label="Go to next page"]'))
        )
        next_btn.click()
        current_page += 1
        time.sleep(2)
    except Exception as e:
        print("❌ 다음 페이지 클릭 실패:", e)
        break

driver.quit()

# ✅ 전체 결과 저장
with open("mypet_all_urls.json", "w", encoding="utf-8") as f:
    json.dump(all_article_urls, f, indent=2, ensure_ascii=False)

print(f"\n✅ 전체 수집 완료! 총 {len(all_article_urls)}개 URL 저장됨 → mypet_all_urls.json")



📄 페이지 1 처리 중...
🔗 기사 URL: https://mypetlife.co.kr/153331/
🔗 기사 URL: https://mypetlife.co.kr/153224/
🔗 기사 URL: https://mypetlife.co.kr/153162/
🔗 기사 URL: https://mypetlife.co.kr/153151/
🔗 기사 URL: https://mypetlife.co.kr/153128/
🔗 기사 URL: https://mypetlife.co.kr/153105/
🔗 기사 URL: https://mypetlife.co.kr/153094/
🔗 기사 URL: https://mypetlife.co.kr/153055/
🔗 기사 URL: https://mypetlife.co.kr/153040/
🔗 기사 URL: https://mypetlife.co.kr/153032/

📄 페이지 2 처리 중...
🔗 기사 URL: https://mypetlife.co.kr/153017/
🔗 기사 URL: https://mypetlife.co.kr/152998/
🔗 기사 URL: https://mypetlife.co.kr/152914/
🔗 기사 URL: https://mypetlife.co.kr/152908/
🔗 기사 URL: https://mypetlife.co.kr/152899/
🔗 기사 URL: https://mypetlife.co.kr/152892/
🔗 기사 URL: https://mypetlife.co.kr/152884/
🔗 기사 URL: https://mypetlife.co.kr/152861/
🔗 기사 URL: https://mypetlife.co.kr/152837/
🔗 기사 URL: https://mypetlife.co.kr/152807/

📄 페이지 3 처리 중...
🔗 기사 URL: https://mypetlife.co.kr/152795/
🔗 기사 URL: https://mypetlife.co.kr/152788/
🔗 기사 URL: https://mypetli

In [31]:
import json
import time
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


def parse_mypetlife_article(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, 'html.parser')

        # 제목
        title_tag = soup.find("h1", class_="entry-title")
        title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

        # 태그
        tag_div = soup.select_one(".post-bottom-tags .tagcloud")
        tags = [a.get_text(strip=True) for a in tag_div.find_all("a")] if tag_div else []

        # 본문
        content_blocks = []
        content_div = soup.find("div", class_="entry-content")
        if not content_div:
            print(f"❌ 본문을 찾을 수 없음: {url}")
            return None

        for elem in content_div.find_all(recursive=True):
            tag = elem.name
            if tag not in {"p", "h2", "h3", "h4", "ul", "ol", "figure"}:
                continue
            if elem.get("class") and any("ads" in cls or "google" in cls or "stream" in cls for cls in elem.get("class")):
                continue
            if elem.find("iframe") or elem.find("noscript"):
                continue

            text = elem.get_text(strip=True)

            if tag == "p" and text:
                content_blocks.append({"type": "paragraph", "text": text})
            elif tag == "h2":
                content_blocks.append({"type": "heading", "text": text})
            elif tag in ["h3", "h4"]:
                if "✔️" in text or "팁" in text:
                    content_blocks.append({"type": "tip_heading", "text": text})
                else:
                    content_blocks.append({"type": "subheading", "text": text})
            elif tag in ["ul", "ol"]:
                items = [li.get_text(strip=True) for li in elem.find_all("li") if li.get_text(strip=True)]
                if items:
                    content_blocks.append({"type": "list", "items": items})
            elif tag == "figure":
                img = elem.find("img")
                if img and img.get("src") and not img["src"].startswith("data:image/svg+xml"):
                    content_blocks.append({"type": "image", "url": img["src"]})

        return {
            "url": url,
            "title": title,
            "tags": tags,
            "content": content_blocks
        }

    except Exception as e:
        print(f"❌ 요청 실패: {url} → {e}")
        return None


# ✅ 전체 URL 리스트 불러오기 및 크롤링 실행
if __name__ == "__main__":
    with open("mypet_all_urls.json", "r", encoding="utf-8") as f:
        urls = json.load(f)

    all_articles = []

    for url in tqdm(urls, desc="📄 기사 수집 중"):
        result = parse_mypetlife_article(url)
        if result:
            all_articles.append(result)
        time.sleep(0.5)  # 서버에 부하 주지 않기 위한 대기

    # ✅ 전체 결과 저장
    with open("mypet_all_articles.json", "w", encoding="utf-8") as f:
        json.dump(all_articles, f, ensure_ascii=False, indent=2)

    print(f"\n✅ 총 {len(all_articles)}개 기사 저장 완료 → mypet_all_articles.json")



📄 기사 수집 중:  50%|████▉     | 633/1270 [06:58<06:41,  1.59it/s]

❌ 본문을 찾을 수 없음: https://mypetlife.co.kr/?post_type=premium#038;p=108023


📄 기사 수집 중:  50%|█████     | 638/1270 [07:01<06:50,  1.54it/s]

❌ 본문을 찾을 수 없음: https://mypetlife.co.kr/?post_type=premium#038;p=107457


📄 기사 수집 중:  51%|█████     | 643/1270 [07:05<06:42,  1.56it/s]

...

❌ 본문을 찾을 수 없음: https://mypetlife.co.kr/?post_type=pauple_helpie#038;p=8054


📄 기사 수집 중:  96%|█████████▌| 1213/1270 [13:46<00:40,  1.41it/s]

❌ 본문을 찾을 수 없음: https://mypetlife.co.kr/?post_type=doglibrary#038;p=6751


📄 기사 수집 중: 100%|██████████| 1270/1270 [14:23<00:00,  1.47it/s]



✅ 총 1114개 기사 저장 완료 → mypet_all_articles.json


### 이미지 url 수집

In [36]:
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# ✅ 1. URL 목록 불러오기 (앞 3개만 테스트용)
with open("mypet_all_urls.json", "r", encoding="utf-8") as f:
    url_list = json.load(f)

test_urls = url_list[:3]  # 테스트용 3개

# ✅ 2. 각 URL에서 대표 이미지(og:image)만 추출
def get_image_url_from_meta(url):
    try:
        res = requests.get(url, timeout=5)
        soup = BeautifulSoup(res.text, "html.parser")
        meta_img = soup.find("meta", property="og:image")
        return meta_img["content"] if meta_img else None
    except Exception as e:
        print(f"❌ 실패: {url} → {e}")
        return None

# ✅ 3. 결과 수집
results = []
for url in tqdm(test_urls, desc="🖼️ 대표 이미지 수집 중"):
    image_url = get_image_url_from_meta(url)
    results.append({
        "url": url,
        "image_url": image_url
    })

# ✅ 4. JSON 파일로 저장 (선택 사항)
with open("mypet_image_urls_test.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("✅ 테스트 이미지 URL 수집 완료 → mypet_image_urls_test.json")


🖼️ 대표 이미지 수집 중: 100%|██████████| 3/3 [00:01<00:00,  2.89it/s]

✅ 테스트 이미지 URL 수집 완료 → mypet_image_urls_test.json


In [37]:
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# ✅ 1. 모든 URL 불러오기
with open("mypet_all_urls.json", "r", encoding="utf-8") as f:
    url_list = json.load(f)

# ✅ 2. 이미지 추출 함수 정의
def get_image_url_from_meta(url):
    try:
        res = requests.get(url, timeout=5)
        if res.status_code != 200:
            return None
        soup = BeautifulSoup(res.text, "html.parser")
        meta_img = soup.find("meta", property="og:image")
        return meta_img["content"] if meta_img else None
    except Exception as e:
        print(f"❌ 실패: {url} → {e}")
        return None

# ✅ 3. 전체 결과 수집
results = []
for url in tqdm(url_list, desc="🖼️ 전체 대표 이미지 수집 중"):
    image_url = get_image_url_from_meta(url)
    results.append({
        "url": url,
        "image_url": image_url
    })

# ✅ 4. 결과 저장
with open("mypet_all_urls_with_image.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 총 {len(results)}개 결과 저장 완료 → mypet_all_urls_with_image.json")


🖼️ 전체 대표 이미지 수집 중: 100%|██████████| 1270/1270 [03:47<00:00,  5.59it/s]


✅ 총 1270개 결과 저장 완료 → mypet_all_urls_with_image.json


In [38]:
import json

# ✅ 1. 이미지 URL 정보 로드
with open("mypet_all_urls_with_image.json", "r", encoding="utf-8") as f:
    url_with_image = json.load(f)

# ✅ 2. 기사 본문 정보 로드
with open("11_mypet_all_articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

# ✅ 3. URL → image_url 매핑 딕셔너리 생성
image_dict = {entry["url"]: entry["image_url"] for entry in url_with_image}

# ✅ 4. 각 기사에 image_url 붙이기
for article in articles:
    url = article["url"]
    article["image_url"] = image_dict.get(url, "")

# ✅ 5. 결과 저장
with open("mypet_all_articles_with_image.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)

print("✅ 병합 완료 → mypet_all_articles_with_image.json 저장됨")


✅ 병합 완료 → mypet_all_articles_with_image.json 저장됨
